In [ ]:
import requests #use for handle requests

In [ ]:
import time #use for set sleep time

In [ ]:
from bs4 import BeautifulSoup #use for parse html files

In [ ]:
import string #use for punctuation

In [ ]:
import json #use for write & read json files

In [ ]:
import pandas as pd #use for create dataframe

In [ ]:
import nltk
#nltk.download("stopwords")
#nltk.download('punkt')
from nltk.corpus import stopwords #use for detect stopwords in string
from nltk.tokenize import word_tokenize #use for getting words in string
from nltk.stem import PorterStemmer #use for stemmer of the word

In [ ]:
#this part is use for getting all movie links in given urls in list
urls = ["https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html",
       "https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html",
        "https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html"
       ] #given urls for getting movies urls
movies_list = []
for url in urls: #for loops for iterate each urls in urls lists
    response = requests.get(url) #get url contact
    soup = BeautifulSoup(response.text, "html.parser")
    for link in soup.find_all("a", href = True): #finding all links and iterate over each one
        link_string = str(link) #changinge link datatype to string
        first_split = link_string.split(">")
        for item in first_split: #iterate over first split items
            if "href" not in item and item != "": #use for finding limks in each of split items
                first_match = item
                second_split = first_match.split("<") #use split for only getting link address
                second_match = second_split[0]
                movies_list.append(second_match) #append link address to movies list
movies_list

In [ ]:
#this part is used to save html files of each movie link
for counter_html in range(len(movies_list)): #iterates over item indexes in movies list
    movie = movies_list[counter_html] #set link by index
    try:
        response = requests.get(movie)
        text = response.text
        file_name_html = "/Users/vahidmohammadi/™️/DS/Practice/project/03/HTML Files/" + str(counter_html) + ".html" #path + file name + file suffix
        with open(file_name_html, "w") as new_movie_file: #open file by above information for saving each file of movie links
            new_movie_file.write(text)
        time.sleep(5)
    except:
        continue

In [ ]:
#this part is used to extract our target information from html files and save them in tsv files
for counter_tsv in range(len(movies_list)):#use for iterate over each html file
    try:
        file_name_html = "/Users/vahidmohammadi/™️/DS/Practice/project/03/HTML Files/" + str(counter_tsv)+".html"#file path of html files
        with open(file_name_html, "r") as movie_html_file: #open  html files for reading
            content = movie_html_file.read() #read html file
            soup = BeautifulSoup(content, "html.parser") #use beautiful soup for parse html contact
            file_name_tsv = "/Users/vahidmohammadi/™️/DS/Practice/project/03/TSV File/" + str(counter_tsv)+".tsv" #file path of tsv files
            with open(file_name_tsv, "w") as movie_tsv_file: #open tsv file for writing
                title = soup.title.string #get title of html file
                intro = soup("p")[0].getText().rstrip("\n") #get intro of html files
                plot = soup("p")[1].getText().rstrip("\n") #get plot of html files
                movie_tsv_file.write(title+"\t"+intro+"\t"+plot+"\t") #writting title, intro and plot in tsv file
                table = soup.find_all("table", class_="infobox vevent")[0]
                second_soup = BeautifulSoup(str(table), "html.parser")
                for item in second_soup.strings:
                    if item.strip("\n")!="":
                        movie_tsv_file.write(item.rstrip("\n")+"\t")
    except:
        continue

In [ ]:
#this part is used for removing stopwords punctuation and get the stem words in tsv files and
#indexing each new words and create dictionary of words exist on movie's tsv file
words = {} #for indexing each word
movies_words = {} #for existance of each word in movie's tsv file
word_id = 0
counter_nltk = 0
exclude = set(string.punctuation)
stop_words = set(stopwords.words('english'))
ps = PorterStemmer() #use for finding stem of each word
for counter_nltk in range(len(movies_list)):
    file_name_tsv = "/Users/vahidmohammadi/™️/DS/Practice/project/03/TSV File/" + str(counter_nltk)+".tsv"#tsv file path
    with open(file_name_tsv, "r") as movie_tsv_file:#open tsv files
        text_list = movie_tsv_file.read()
        text = " ".join(text_list.split("\t")[1:3])
        text = ''.join(ch for ch in text if ch not in exclude) #use for getting each word without white spaces
        word_tokens = word_tokenize(text)
        filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]#removing stopwords and finding stem of the each word
        for word in filtered_sentence:
            if word not in words.values():#looking for NOT existance of word in words list for indexing
                words[word_id] = word
                movies_words[word_id] = [counter_nltk]#adding movies name in movies_words
                word_id += 1
            else: # if word exist in words list just add movies ID (movies file name that produce by counter) in movies word
                exist_id = [key for key, value in words.items() if value == word][0]
                if counter_nltk not in movies_words[exist_id]:
                    movies_words[exist_id].append(counter_nltk)
with open("words_index.json", "w") as word_index: #write words list in json file
    json.dump(words, word_index)
with open("movies_words.json", "w") as movies_words_file:#write movies_words list in json file
    json.dump(movies_words, movies_words_file)

In [ ]:
print("==============================")
print("|What movie to watch tonight?|")
print("==============================")
print("--------------------")
print("|Search Engine v1.0|")
print("--------------------")
our_input = input("search: ")
exclude = set(string.punctuation)
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
our_input = ''.join(ch for ch in our_input if ch not in exclude)
word_tokens = word_tokenize(our_input)
filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]
words_in_input = []
with open("words_index.json", "r") as word_index:
    words = json.load(word_index)
with open("movies_words.json", "r") as movies_words_file:
    movies_words = json.load(movies_words_file)
for word in filtered_sentence:
    for key, value in words.items():
        if word == value:
            words_in_input.append(key)
words_in_movies = []
for key_of_input in words_in_input:#finding each searched word's index
    for key_of_movies, value in movies_words.items():
        if key_of_input == key_of_movies:
            words_in_movies.append(value)
final_result_dict = {}#use for saving matches
for first_item in words_in_movies:#searching by words index that exist in movies words list
    for second_item in first_item:
        if second_item in final_result_dict.keys():
            final_result_dict[second_item] += 1
        else:
            final_result_dict[second_item] = 1
all_item_match_dict = {}
for key, value in final_result_dict.items():
    if value == len(words_in_input):
        all_item_match_dict[key] = value
print("")
print("Result: ")
df = {"movie":[], "intro":[], "link":[]}#use for creating pandas dataframe
for item in all_item_match_dict:
    file_name = "/Users/vahidmohammadi/™️/DS/Practice/project/03/TSV File/" + str(item) + ".tsv"#tsv file path
    with open(file_name, "r") as target:#open tsv file for reading title and intro
        tsv_file = target.read()
        movie_list = tsv_file.split("\t")
        df["movie"].append(movie_list[0])
        df["intro"].append(movie_list[1])
        df["link"].append(movies_list[item])
pd_df = pd.DataFrame(df)
if pd_df.empty:
    print("NOT Found 404")
pd_df.head(5)